# 初始化

In [1]:
from qlib.contrib.data.handler import Alpha158
def qlib_init():
    import qlib 
    from qlib.config import REG_CN
    provider_uri = "~/.qlib/qlib_data/cn_data"  # target_dir
    qlib.init(provider_uri=provider_uri, region=REG_CN)

def load_alpha158_handler(start_time,end_time,valid_end,market):
    from qlib.contrib.data.handler import Alpha158
    config = {
        "start_time": start_time,
        "end_time": end_time,
        "fit_start_time": start_time,
        "fit_end_time": valid_end,
        "instruments": market,
    }
    return Alpha158(**config)

# 参数设置

In [2]:
#训练时间设置
start_time = "2015-07-01" # 数据开始时间
end_time =  "2020-12-31" # 数据结束时间/测试结束时间/回测结束时间
train_end = "2019-06-30" # 训练结束时间
valid_start = "2019-07-01" # 验证开始时间
valid_end = "2020-06-30" # 验证结束时间
test_start = "2020-07-01" # 测试开始时间/回测开始时间

train_segments = {
                "train": (start_time, train_end),
                "valid": (valid_start, valid_end),
                "test": (test_start, end_time),
            }

#市场设置
market = "yunnan"
instruments = market

qlib_init()# qlib 初始化

[22960:MainThread](2023-03-14 10:01:44,183) INFO - qlib.Initialization - [config.py:413] - default_conf: client.
[22960:MainThread](2023-03-14 10:01:44,940) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[22960:MainThread](2023-03-14 10:01:44,944) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/is_li/.qlib/qlib_data/cn_data')}


In [3]:
handler = load_alpha158_handler(start_time,end_time,valid_end,market)
df = handler.fetch()# 因子数据表

In [25]:


from qlib.data.dataset import DatasetH
d = DatasetH(load_alpha158_handler(),segments=train_segments)#dataset

from qlib.data.dataset.loader import QlibDataLoader
MACD_EXP = '(EMA($close, 12) - EMA($close, 26))/$close - EMA((EMA($close, 12) - EMA($close, 26))/$close, 9)/$close'
mom_5 = '$close/Ref($close, 5) - 1'
fields = [MACD_EXP, '$close',mom_5]  # MACD
names = ['MACD', '收盘价','五日动量']
labels = ['Ref($close, -1)/$close - 1','Ref($close, -2)/Ref($close, -1) - 1']  # label
label_names = ['LABEL_1', 'LABEL_2']
data_loader_config = {
    "feature": (fields, names),
    "label": (labels, label_names)
}
data_loader = QlibDataLoader(config=data_loader_config)

from ..code_lib.parameter_setting import load_lightGBM
model = load_lightGBM()
dataset = DatasetH(load_alpha158_handler(),segments=train_segments)

NameError: name 'load_alpha158_handler' is not defined